# Chapter 2 — Parallel Programming with Threads (Extracted Code)

This notebook collects runnable Python examples extracted from the chapter.
Each section mirrors the examples shown in the text, cleaned and made executable.


## 1. Basic `threading.Thread` without `.join()`

In [ ]:
import threading
import time

def function(i):
    print(f"start Thread {i}")
    time.sleep(2)
    print(f"end Thread {i}")

t1 = threading.Thread(target=function, args=(1,))
t2 = threading.Thread(target=function, args=(2,))
t3 = threading.Thread(target=function, args=(3,))
t4 = threading.Thread(target=function, args=(4,))
t5 = threading.Thread(target=function, args=(5,))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()

print("END Program")


## 2. Waiting for all threads with `.join()`

In [ ]:
import threading
import time

def function(i):
    print(f"start Thread {i}")
    time.sleep(2)
    print(f"end Thread {i}")

t1 = threading.Thread(target=function, args=(1,))
t2 = threading.Thread(target=function, args=(2,))
t3 = threading.Thread(target=function, args=(3,))
t4 = threading.Thread(target=function, args=(4,))
t5 = threading.Thread(target=function, args=(5,))

t1.start(); t2.start(); t3.start(); t4.start(); t5.start()

t1.join(); t2.join(); t3.join(); t4.join(); t5.join()
print("END Program")


## 3. Joining in phases to synchronize parts of the program

In [ ]:
import threading
import time

def function(i):
    print(f"start Thread {i}")
    time.sleep(2)
    print(f"end Thread {i}")

t1 = threading.Thread(target=function, args=(1,))
t2 = threading.Thread(target=function, args=(2,))
t3 = threading.Thread(target=function, args=(3,))
t4 = threading.Thread(target=function, args=(4,))
t5 = threading.Thread(target=function, args=(5,))

# First phase
t1.start(); t2.start()
t1.join(); t2.join()
print("First set of threads done")
print("The program can execute other code here")

# Second phase
t3.start(); t4.start(); t5.start()
t3.join(); t4.join(); t5.join()
print("Second set of threads done")
print("END Program")


## 4. Common pattern: create/join in loops

In [ ]:
import threading
import time

def function(i):
    print(f"start Thread {i}")
    time.sleep(2)
    print(f"end Thread {i}")

n_threads = 5
threads = []

for i in range(n_threads):
    t = threading.Thread(target=function, args=(i,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()


## 5. `concurrent.futures.ThreadPoolExecutor` with a pool of 3

In [ ]:
import concurrent.futures
import time
import random
import threading

def task(i, sleep_s):
    tname = threading.current_thread().name
    print(f"[{time.strftime('%H:%M:%S')}] START task={i:02d} sleep={sleep_s:.1f}s on {tname}")
    time.sleep(sleep_s)
    print(f"[{time.strftime('%H:%M:%S')}] END   task={i:02d} on {tname}")
    return i, sleep_s

durations = [1.5, 0.8, 2.2, 0.5, 1.0, 1.7, 0.6, 2.0, 0.9, 1.3]

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(task, i, d) for i, d in enumerate(durations, start=1)]
    for fut in concurrent.futures.as_completed(futures):
        i, d = fut.result()

print("Program ended")


## 6. Competition on shared string (race condition demonstration)

In [ ]:
import threading
import time
import random

sequence = ""
COUNT = 10

def addA():
    global sequence
    for _ in range(COUNT):
        time.sleep(random.uniform(0.1, 0.3))
        sequence = f"{sequence}A"
        print("Sequence:", sequence)

def addB():
    global sequence
    for _ in range(COUNT):
        time.sleep(random.uniform(0.1, 0.3))
        sequence = f"{sequence}B"
        print("Sequence:", sequence)

t1 = threading.Thread(target=addA)
t2 = threading.Thread(target=addB)
t1.start(); t2.start()
t1.join(); t2.join()


## 7. Subclassing `Thread`: template

In [ ]:
from threading import Thread

class MyThread(Thread):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # custom initialization here

    def run(self):
        # thread logic here
        pass

t = MyThread()
t.start()
t.join()


## 8. Subclassing `Thread` with two concrete workers (A/B)

In [ ]:
from threading import Thread
import time

sequence = ""
COUNT = 5
timeA = 1
timeB = 2

class ThreadA(Thread):
    def run(self):
        global sequence
        for _ in range(COUNT):
            time.sleep(timeA)
            sequence += "A"
            print("Sequence:", sequence)

class ThreadB(Thread):
    def run(self):
        global sequence
        for _ in range(COUNT):
            time.sleep(timeB)
            sequence += "B"
            print("Sequence:", sequence)

t1 = ThreadA()
t2 = ThreadB()
t1.start(); t2.start()
t1.join(); t2.join()


## 9. Race condition on integer without synchronization

In [ ]:
import threading, time

shared_data = 0

def funcA():
    global shared_data
    for i in range(10_000):
        v = shared_data
        time.sleep(0)
        shared_data = v + 10
        if i % 2000 == 0:
            print(f"A i={i:5d} -> {shared_data}")

def funcB():
    global shared_data
    for i in range(10_000):
        v = shared_data
        time.sleep(0)
        shared_data = v - 10
        if i % 2000 == 0:
            print(f"B i={i:5d} -> {shared_data}")

t1 = threading.Thread(target=funcA)
t2 = threading.Thread(target=funcB)
t1.start(); t2.start()
t1.join(); t2.join()

print("Final (unsynchronized):", shared_data)


## 10. Fixing race with `Lock` and context manager

In [ ]:
import threading, time

shared_data = 0
lock = threading.Lock()

def funcA():
    global shared_data
    for i in range(10_000):
        with lock:
            shared_data += 10
            if i % 2000 == 0:
                print(f"A i={i:5d} -> {shared_data}")
        time.sleep(0)

def funcB():
    global shared_data
    for i in range(10_000):
        with lock:
            shared_data -= 10
            if i % 2000 == 0:
                print(f"B i={i:5d} -> {shared_data}")
        time.sleep(0)

t1 = threading.Thread(target=funcA)
t2 = threading.Thread(target=funcB)
t1.start(); t2.start()
t1.join(); t2.join()

print("Final (synchronized):", shared_data)


## 11. `with lock:` pattern (guaranteed release)

In [ ]:
import threading
import time

shared_data = 0
lock = threading.Lock()

def funcA():
    global shared_data
    for i in range(10):
        with lock:
            local = shared_data
            local += 10
            time.sleep(1)
            shared_data = local
            print("Thread A wrote:", shared_data)

def funcB():
    global shared_data
    for i in range(10):
        with lock:
            local = shared_data
            local -= 10
            time.sleep(1)
            shared_data = local
            print("Thread B wrote:", shared_data)

t1 = threading.Thread(target=funcA)
t2 = threading.Thread(target=funcB)
t1.start(); t2.start()
t1.join(); t2.join()


## 12. Asymmetric acquire/release (didactic, not recommended)

In [ ]:
# WARNING: This example is intentionally asymmetric and only for debugging/didactic purposes.
import threading
import time

shared = 0
lock = threading.Lock()

def funcA():
    global shared
    for i in range(10):
        time.sleep(1)
        shared += 10
        print(f"Thread A wrote: {shared}, {i}")
        lock.acquire()

def funcB():
    global shared
    lock.acquire()
    for i in range(10):
        time.sleep(1)
        shared -= 10
        print(f"Thread B wrote: {shared}, {i}")
        lock.release()

t1 = threading.Thread(target=funcA)
t2 = threading.Thread(target=funcB)
t1.start(); t2.start()
t1.join(); t2.join()


## 13. Reentrant locks (`RLock`) and nested acquire

In [ ]:
import threading
import time

shared = 0
rlock = threading.RLock()

def func(name, t):
    global shared
    for _ in range(3):
        rlock.acquire()
        local = shared
        time.sleep(t)
        for j in range(2):
            rlock.acquire()
            local += 1
            time.sleep(0.2)
            shared = local
            print(f"Thread {name}-{j} wrote: {shared}")
            rlock.release()
        shared = local + 1
        print(f"Thread {name} wrote: {shared}")
        rlock.release()

t1 = threading.Thread(target=func, args=('A', 0.2))
t2 = threading.Thread(target=func, args=('B', 0.1))
t3 = threading.Thread(target=func, args=('C', 0.05))
t1.start(); t2.start(); t3.start()
t1.join(); t2.join(); t3.join()

print("Expected final value:", shared, "(should be 27)")


## 14. Semaphore-based producer–consumer (single-slot)

In [ ]:
import threading
import time
import random

N_ITEMS = 5
slots = threading.Semaphore(1)
items = threading.Semaphore(0)

buffer = {"value": None}
print_lock = threading.Lock()

def producer():
    for i in range(1, N_ITEMS + 1):
        slots.acquire()
        v = random.randint(0, 100)
        time.sleep(random.uniform(0.2, 0.6))
        buffer["value"] = v
        with print_lock:
            print(f"producer -> put {v:3d} (item {i})")
        items.release()

def consumer():
    for i in range(1, N_ITEMS + 1):
        items.acquire()
        v = buffer["value"]
        time.sleep(random.uniform(0.1, 0.5))
        with print_lock:
            print(f"consumer <- got {v:3d} (item {i})")
        buffer["value"] = None
        slots.release()

tp = threading.Thread(target=producer, name="Producer")
tc = threading.Thread(target=consumer, name="Consumer")
tp.start(); tc.start()
tp.join(); tc.join()
print("Done.")


## 15. Condition variables: producer–consumer alternation

In [ ]:
from threading import Thread, Condition
import time, random

condition = Condition()
shared = None
count = 5

class Consumer(Thread):
    def __init__(self, count):
        super().__init__()
        self.count = count
    
    def run(self):
        global shared
        for _ in range(self.count):
            with condition:
                while shared is None:
                    condition.wait()
                print(f"consumer <- got {shared}")
                shared = None
                condition.notify()

class Producer(Thread):
    def __init__(self, count):
        super().__init__()
        self.count = count

    def run(self):
        global shared
        for _ in range(self.count):
            time.sleep(1)
            item = random.randint(0, 100)
            with condition:
                while shared is not None:
                    condition.wait()
                shared = item
                print(f"producer -> put {item}")
                condition.notify()

t1 = Producer(count)
t2 = Consumer(count)
t1.start(); t2.start()
t1.join(); t2.join()


## 16. Event-based handoff (single producer & consumer)

In [ ]:
from threading import Thread, Event
import time, random

event = Event()
shared = 1
count = 5

class Consumer(Thread):
    def __init__(self, count):
        super().__init__()
        self.count = count
  
    def run(self):
        global shared
        for _ in range(self.count):
            event.wait()
            print("consumer has used this:", shared)
            shared = 0
            event.clear()

class Producer(Thread):
    def __init__(self, count):
        super().__init__()
        self.count = count

    def request(self):
        time.sleep(1)
        return random.randint(0, 100)
 
    def run(self):
        global shared
        for _ in range(self.count):
            shared = self.request()
            print("producer has loaded this:", shared)
            event.set()

t1 = Producer(count)
t2 = Consumer(count)
t1.start(); t2.start()
t1.join(); t2.join()


## 17. Why `Event` breaks with multiple producers/consumers

In [ ]:
# This example demonstrates why Event is not suitable for coordinating multiple producers/consumers.
from threading import Thread, Event
import time, random

event = Event()
shared = 1
count = 5

class Consumer(Thread):
    def __init__(self, count):
        super().__init__()
        self.count = count
  
    def run(self):
        global shared
        for _ in range(self.count):
            event.wait()
            print("consumer has used this:", shared)
            shared = 0
            event.clear()

class Producer(Thread):
    def __init__(self, count):
        super().__init__()
        self.count = count

    def request(self):
        time.sleep(1)
        return random.randint(0, 100)
 
    def run(self):
        global shared
        for _ in range(self.count):
            shared = self.request()
            print("producer has loaded this:", shared)
            event.set()

t1 = Producer(count); t2 = Producer(count)
t3 = Consumer(count); t4 = Consumer(count)
for t in (t1, t2, t3, t4): t.start()
for t in (t1, t2, t3, t4): t.join()


## 18. Queue-based multi-producer/multi-consumer (correct)

In [ ]:
from threading import Thread
from queue import Queue
import time, random

queue = Queue()
count = 5

class Consumer(Thread):
    def __init__(self, count):
        super().__init__()
        self.count = count
  
    def run(self):
        for _ in range(self.count):
            local = queue.get()
            print("consumer has used this:", local)
            queue.task_done()

class Producer(Thread):
    def __init__(self, count):
        super().__init__()
        self.count = count

    def request(self):
        time.sleep(1)
        return random.randint(0, 100)
 
    def run(self):
        for _ in range(self.count):
            local = self.request()
            queue.put(local)
            print("producer has loaded this:", local)

t1 = Producer(count); t2 = Producer(count)
t3 = Consumer(count); t4 = Consumer(count)
for t in (t1, t2, t3, t4): t.start()
for t in (t1, t2, t3, t4): t.join()
